In [18]:
%matplotlib inline
%load_ext autoreload

from __future__ import division

import numpy as np
import os
import mne
import pandas as pd

from scipy import io

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
home = os.environ["HOME"]
data_root = home+"/data/karaOne/"

data_dirs = [data_root+f+'/' for f in os.listdir(data_root)
            if f[:2]=='MM']

mapper = lambda root: [root+f for f in os.listdir(root) if f[-4:]=='.cnt'][0]
cnt_files = sorted(map(mapper, data_dirs))

mapper = lambda root: [root+f for f in os.listdir(root) if f=='epoch_inds.mat'][0]
ind_files = sorted(map(mapper, data_dirs))

lbl_files = lambda root: [root+f for f in os.listdir(root) if f=='.txt'][0]
ind_files = sorted(map(mapper, data_dirs))

for c in ind_files:
    print c

/home/UFAD/yliapis/data/karaOne/MM05/epoch_inds.mat
/home/UFAD/yliapis/data/karaOne/MM08/epoch_inds.mat
/home/UFAD/yliapis/data/karaOne/MM09/epoch_inds.mat
/home/UFAD/yliapis/data/karaOne/MM10/epoch_inds.mat
/home/UFAD/yliapis/data/karaOne/MM11/epoch_inds.mat
/home/UFAD/yliapis/data/karaOne/MM12/epoch_inds.mat
/home/UFAD/yliapis/data/karaOne/MM14/epoch_inds.mat
/home/UFAD/yliapis/data/karaOne/MM15/epoch_inds.mat
/home/UFAD/yliapis/data/karaOne/MM16/epoch_inds.mat
/home/UFAD/yliapis/data/karaOne/MM18/epoch_inds.mat
/home/UFAD/yliapis/data/karaOne/MM19/epoch_inds.mat
/home/UFAD/yliapis/data/karaOne/MM20/epoch_inds.mat
/home/UFAD/yliapis/data/karaOne/MM21/epoch_inds.mat


In [52]:
print d.keys()
print d[d.keys()[5]]

['thinking_inds', '__header__', '__globals__', 'clearing_inds', 'speaking_inds', '__version__']
1.0


In [47]:
d = io.loadmat(ind_files[0])
print d[d.keys()[3]].flatten()[0]
dd = d[d.keys()[0]]
a,b = d[d.keys()[0]].flatten()[0].flatten()

[[ 8233 13195]]


In [44]:
ddd = np.array(dd.tolist())
ddd.shape

(1, 165, 1, 2)

In [28]:
d['clearing_inds']
label_dict = {'clearing_inds': 0, 'thinking_inds': 1, 'speaking_inds':2}

for k,y in label_dict.items():
    print k,y

thinking_inds 1
clearing_inds 0
speaking_inds 2


In [56]:
#fdt = data_dirs[0]+'set_files/Acqusition 232 Data.fdt'
cnt_file = cnt_files[1]
f1 = mne.io.read_raw_cnt(cnt_file, None, date_format="dd/mm/yy")
f1.ch_names

['FP1',
 'FPZ',
 'FP2',
 'AF3',
 'AF4',
 'F7',
 'F5',
 'F3',
 'F1',
 'FZ',
 'F2',
 'F4',
 'F6',
 'F8',
 'FT7',
 'FC5',
 'FC3',
 'FC1',
 'FCZ',
 'FC2',
 'FC4',
 'FC6',
 'FT8',
 'T7',
 'C5',
 'C3',
 'C1',
 'CZ',
 'C2',
 'C4',
 'C6',
 'T8',
 'M1',
 'TP7',
 'CP5',
 'CP3',
 'CP1',
 'CPZ',
 'CP2',
 'CP4',
 'CP6',
 'TP8',
 'M2',
 'P7',
 'P5',
 'P3',
 'P1',
 'PZ',
 'P2',
 'P4',
 'P6',
 'P8',
 'PO7',
 'PO5',
 'PO3',
 'POZ',
 'PO4',
 'PO6',
 'PO8',
 'CB1',
 'O1',
 'OZ',
 'O2',
 'CB2',
 'VEO',
 'HEO',
 'EKG',
 'EMG',
 'Trigger',
 'STI 014']

In [12]:

EEG_series = []
for f in cnt_files:
    EEG_series.append(mne.io.read_raw_cnt(f, None, date_format="dd/mm/yy").to_data_frame())
    
    

Converting time column to int64...
Converting time column to int64...
Converting time column to int64...
Converting time column to int64...
Converting time column to int64...
Converting time column to int64...
Converting time column to int64...
Converting time column to int64...
Converting time column to int64...
Converting time column to int64...
Converting time column to int64...
Converting time column to int64...
Converting time column to int64...


In [15]:
def process_dataframe(df):
    return df.iloc[:,:64].values

In [17]:
print process_dataframe(data).dtype

float64


In [58]:
def import_data(ret_metadata=False):
    # define root, filepath
    home = os.environ["HOME"]
    data_root = home+"/data/karaOne/"
    # get files
    data_dirs = [data_root+f+'/' for f in os.listdir(data_root)
            if f[:2]=='MM']
    #X files
    mapper = lambda root: [root+f for f in os.listdir(root) if f[-4:]=='.cnt'][0]
    cnt_files = sorted(map(mapper, data_dirs))
    #Y files
    mapper = lambda root: [root+f for f in os.listdir(root) if f=='epoch_inds.mat'][0]
    ind_files = sorted(map(mapper, data_dirs))
    # process X
    channel_range = np.arange(0,64)
    channel_dict = {name:i for i,name in enumerate(
            mne.io.read_raw_cnt(f, None, date_format="dd/mm/yy").ch_names[:len(channel_range)])}
    def m0(f):
        obj = mne.io.read_raw_cnt(f, None, date_format="dd/mm/yy")
        df = obj.to_data_frame()
        return df.iloc[:,channel_range]
    X_series = map(m0, cnt_files)
    # process Y
    label_dict = {'clearing_inds': 0, 'thinking_inds': 1, 'speaking_inds':2}
    Y_series = []
    for X,f in zip(X_series, ind_files):
        ind_dict = io.loadmat(f)
        Y = np.zeroes(X.shape[0])
        for key,val in label_dict.items():
            label = label_dict[key]
            #fix dict
            ind_dict[key] = np.array(ind_dict[key].tolist()).flatten()
            #
            for ind0,indf in ind_dict[key]:
                Y[ind0-1:indf] = label
        Y_series.append(Y)
    #
    if ret_metadata:
        metadata = {'labels': label_dict, 'channels': channel_dict}
        return X_series, Y_series, metadata
    else:
        return X_series, Y_series
    
    
    

In [14]:
for i,c in enumerate(data.columns):
    print i,c

0 FP1
1 FPZ
2 FP2
3 AF3
4 AF4
5 F7
6 F5
7 F3
8 F1
9 FZ
10 F2
11 F4
12 F6
13 F8
14 FT7
15 FC5
16 FC3
17 FC1
18 FCZ
19 FC2
20 FC4
21 FC6
22 FT8
23 T7
24 C5
25 C3
26 C1
27 CZ
28 C2
29 C4
30 C6
31 T8
32 M1
33 TP7
34 CP5
35 CP3
36 CP1
37 CPZ
38 CP2
39 CP4
40 CP6
41 TP8
42 M2
43 P7
44 P5
45 P3
46 P1
47 PZ
48 P2
49 P4
50 P6
51 P8
52 PO7
53 PO5
54 PO3
55 POZ
56 PO4
57 PO6
58 PO8
59 CB1
60 O1
61 OZ
62 O2
63 CB2
64 VEO
65 HEO
66 EKG
67 EMG
68 Trigger
69 STI 014
